<a href="https://colab.research.google.com/github/louisaliu173-glitch/QM2.0/blob/main/project1_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/Thailand Data.csv",
    encoding="utf-8"
)

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,COUNTRY,CITY,YEAR,TEMPERATURE,GDP PER CAPITA,POPULATION,LATITUDE,LONGITUDE,ELEVATION
0,Thailand,Bangkok,2016,28.49,17364.0,6150.315548,13.6585 N,99.9083E,1.5
1,Thailand,Bangkok,2017,28.42,18266.3,6310.320648,13.6585 N,99.9083E,1.5
2,Thailand,Bangkok,2018,28.75,19299.4,6474.150571,13.6585 N,99.9083E,1.5
3,Thailand,Bangkok,2019,28.76,20267.5,6597.819851,13.6585 N,99.9083E,1.5
4,Thailand,Bangkok,2020,28.84,18762.2,6718.301779,13.6585 N,99.9083E,1.5


In [18]:
import numpy as np

df = df.sort_values(['CITY', 'YEAR'])

def fill_linear(series, years):
    known = series.dropna()
    if len(known) < 2:
        return series
    y_known = years.loc[known.index]
    slope = (known.iloc[-1] - known.iloc[0]) / (y_known.iloc[-1] - y_known.iloc[0])
    intercept = known.iloc[0] - slope * y_known.iloc[0]
    out = series.copy()
    out.loc[series.isna()] = slope * years.loc[series.isna()] + intercept
    return out

df['GDP PER CAPITA'] = (
    df.groupby('CITY', group_keys=False)
      .apply(lambda g: fill_linear(g['GDP PER CAPITA'], g['YEAR']))
)

df['POPULATION'] = (
    df.groupby('CITY', group_keys=False)
      .apply(lambda g: fill_linear(g['POPULATION'], g['YEAR']))
)

df['TEMPERATURE'] = (
    df.groupby('CITY')['TEMPERATURE']
      .transform(lambda x: x.fillna((x.shift() + x.shift(-1)) / 2))
)

df['TEMPERATURE'] = (
    df.groupby('CITY', group_keys=False)
      .apply(lambda g: fill_linear(g['TEMPERATURE'], g['YEAR']))
)

df.head()
df.to_csv("Vietnam_Data_cleaned.csv", index=False)

for col in ['TEMPERATURE', 'GDP PER CAPITA', 'POPULATION']:
    df[col] = df[col].round(2)
print(df)


     COUNTRY        CITY  YEAR  TEMPERATURE  GDP PER CAPITA  POPULATION  \
0   Thailand     Bangkok  2016        28.49        17364.00     6150.32   
1   Thailand     Bangkok  2017        28.42        18266.30     6310.32   
2   Thailand     Bangkok  2018        28.75        19299.40     6474.15   
3   Thailand     Bangkok  2019        28.76        20267.50     6597.82   
4   Thailand     Bangkok  2020        28.84        18762.20     6718.30   
..       ...         ...   ...          ...             ...         ...   
45  Thailand  Udon Thani  2021        27.90         6395.12      544.85   
46  Thailand  Udon Thani  2022        27.40         6747.01      550.52   
47  Thailand  Udon Thani  2023        27.40         6751.37      557.13   
48  Thailand  Udon Thani  2024        27.60         6821.27      563.74   
49  Thailand  Udon Thani  2025        27.60         6891.17      570.35   

     LATITUDE    LONGITUDE  ELEVATION  
0   13.6585 N     99.9083E        1.5  
1   13.6585 N     9

/tmp/ipython-input-564706785.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_linear(g['GDP PER CAPITA'], g['YEAR']))
/tmp/ipython-input-564706785.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_linear(g['POPULATION'], g['YEAR']))
/tmp/ipython-input-564706785.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior